In [ ]:
!pip install openai==0.28
!pip uninstall openai
!pip install openai==0.28
import openai
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.10/dist-packages/openai-0.28.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/openai/*
Proceed (Y/n)? Y
  Successfully uninstalled openai-0.28.0
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)


In [ ]:
#Loading the dataset

file_path = "/content/student_admission_dataset.csv"
dataset = pd.read_csv(file_path)
dataset.head()

,GPA,SAT_Score,Extracurricular_Activities,Admission_Status
0,3.46,1223,8,Rejected
1,2.54,974,8,Rejected
2,2.91,909,9,Rejected
3,2.83,1369,5,Accepted
4,3.60,1536,7,Accepted


In [ ]:
#Encoding admission status as a binary variable 1 or 0
dataset['Admission_Status'] = dataset['Admission_Status'].apply(lambda x: 1 if x == 'Accepted' else 0)


In [ ]:
#Defining the features and the target variable
X = dataset.drop('Admission_Status', axis=1)
y = dataset['Admission_Status']

In [ ]:
#Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Separating into accepted and rejected subsets
accepted = dataset[dataset['Admission_Status'] == 1]
rejected = dataset[dataset['Admission_Status'] == 0]


In [ ]:
#Oversampling accepted because it's the minority
accepted_oversampled = accepted.sample(len(rejected), replace=True, random_state=42)


In [ ]:
#Combining oversampled to original rejected pool to balance the dataset
balanced_dataset = pd.concat([rejected, accepted_oversampled])


In [ ]:
#Shuffling
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
#Separating the balanced dataset
X_balanced = balanced_dataset.drop('Admission_Status', axis=1)
y_balanced = balanced_dataset['Admission_Status']


In [ ]:
#Splitting into 80% training 20% test
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [ ]:
#initializing rfc (RANDOM FOREST CLASSIFIER)
rf_model_balanced = RandomForestClassifier(random_state=42)
rf_model_balanced.fit(X_train_balanced, y_train_balanced)

RandomForestClassifier(random_state=42)

In [ ]:
#function for predicting admission status using LLM
def predict_admission_integrated(gpa, sat_score, extracurricular_activities):
    user_data = pd.DataFrame([[gpa, sat_score, extracurricular_activities]], columns=['GPA', 'SAT_Score', 'Extracurricular_Activities'])
    print(user_data.head())

    rf_prediction = rf_model_balanced.predict(user_data)[0]
    rf_prediction_text = 'Accepted' if rf_prediction == 1 else 'Rejected'

    input_text = (f"GPA: {gpa}\n"
                  f"SAT Score: {sat_score}\n"
                  f"Extracurricular Activities: {extracurricular_activities}\n"
                  f"The Random Forest model predicts: {rf_prediction_text}\n"
                  f"Based on this data, would the student likely be accepted or rejected for admission? Please answer with 'Accepted' or 'Rejected'.")
    # Call the LLM API using ChatCompletion API
    openai.api_key = 'sk-proj-amAbPYXB8hJZwvBYxLtQT3BlbkFJARmJd86E0H0jBBdGJorA'
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": input_text}],
        max_tokens=30,
        temperature=0
    )
    prediction = response.choices[0].message['content'].strip()
    if 'accepted' in prediction.lower():
      return 'Accepted'
    elif "rejected" in prediction.lower():
      return 'Rejected'
    else:
      return 'Undetermined'

In [ ]:
gpa = float(input("Enter your GPA: "))
sat_score = int(input("Enter your SAT Score: "))
extracurricular_activities = int(input("Enter the number of extracurricular activities: "))

result = predict_admission_integrated(gpa, sat_score, extracurricular_activities)
print("Prediction: You are likely to be", result)

Enter your GPA: 3.9
Enter your SAT Score: 1510
Enter the number of extracurricular activities: 10
   GPA  SAT_Score  Extracurricular_Activities
0  3.9       1510                          10
Prediction: You are likely to be Accepted
